# MANIPULAÇÃO DOS DADOS

## Sessão Spark

In [3]:
import findspark
import os

findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Spark csv schema inference") \
    .config("spark.sql.warehouse.dir") \
    .enableHiveSupport() \
    .getOrCreate()

## Imports

In [6]:
from pyspark.sql.functions import *
import pandas as pd
import sqlite3
from functools import reduce

## PARÂMETROS DO NOTEBOOK

In [7]:
# Caminho dos CSVs
sales_data_path = 'data/sales_data.csv'
website_data_path = 'data/website_logs.csv'

## Carregando dados de CSV

In [8]:
sales = spark.read.csv(sales_data_path, sep = ',', header = True)
website = spark.read.csv(website_data_path, sep = ',', header = True)

## Visualizando dados

In [9]:
sales.show(5)

+--------------+----------+----------+---------+----------+--------+
|transaction_id|      date|product_id|seller_id|sale_value|currency|
+--------------+----------+----------+---------+----------+--------+
|          1001|2023-07-25|       501|     2001|       150|    FICT|
|          1002|2023-07-25|       502|     2001|       300|    FICT|
|          1003|2023-07-25|       501|     2002|       160|    FICT|
|          1004|2023-07-25|       503|     2002|       210|    FICT|
|          1005|2023-07-25|       504|     2003|        75|    FICT|
+--------------+----------+----------+---------+----------+--------+
only showing top 5 rows



In [10]:
website.show(5)

+-------+-----------------+----------------+----------+
|user_id|         page_url|session_duration|      date|
+-------+-----------------+----------------+----------+
|  10001|    homepage.html|              15|2023-07-25|
|  10002|product_page.html|             120|2023-07-25|
|  10003|    checkout.html|              45|2023-07-25|
|  10004|     contact.html|              20|2023-07-25|
|  10005|    homepage.html|              10|2023-07-25|
+-------+-----------------+----------------+----------+
only showing top 5 rows



## Contagens

In [11]:
print('count sales_data: ', sales.count())
print('count website_logs: ', website.count())

count sales_data:  40
count website_logs:  184


## Schema

In [17]:
print(sales.printSchema())
print(website.printSchema())

root
 |-- transaction_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- sale_value: string (nullable = true)
 |-- currency: string (nullable = true)

None
root
 |-- user_id: string (nullable = true)
 |-- page_url: string (nullable = true)
 |-- session_duration: string (nullable = true)
 |-- date: string (nullable = true)

None


## Limpando os dados removendo linhas duplicadas e tratando valores ausentes

In [18]:
sales_clean = sales.distinct()
website_clean = website.distinct()

print('count sales_data LIMPO: ', sales_clean.count())
print('count website_logs LIMPO: ', website_clean.count())

count sales_data LIMPO:  40
count website_logs LIMPO:  184


#### Contagem de Nulos por coluna

In [27]:
print('sales_data')
print('Quantidade de nulos por colunas:')
sales_clean.select([count(when(col(c).isNull(),c)).alias(c) for c in sales_clean.columns]).show()

sales_data
Quantidade de nulos por colunas:
+--------------+----+----------+---------+----------+--------+
|transaction_id|date|product_id|seller_id|sale_value|currency|
+--------------+----+----------+---------+----------+--------+
|             0|   0|         0|        0|         0|       0|
+--------------+----+----------+---------+----------+--------+



In [26]:
print('website_data')
print('Quantidade de nulos por colunas:')
website_clean.select([count(when(col(c).isNull(),c)).alias(c) for c in website_clean.columns]).show()

website_data
Quantidade de nulos por colunas:
+-------+--------+----------------+----+
|user_id|page_url|session_duration|date|
+-------+--------+----------------+----+
|      0|       0|               0|   0|
+-------+--------+----------------+----+



### Não há valores ausentes ou duplicações!

## Transformando o valor da venda de uma moeda fictícia para USD usando a taxa de conversão de 1 FICT = 0.75 USD.

In [41]:
sales_converted = sales_clean\
    .withColumn('sale_value', when(col('currency') == 'FICT', col('sale_value') * 0.75))\
    .withColumn('currency', when(col('currency') == 'FICT', lit('USD')))

In [42]:
# Visualizando
sales_converted.show()

+--------------+----------+----------+---------+----------+--------+
|transaction_id|      date|product_id|seller_id|sale_value|currency|
+--------------+----------+----------+---------+----------+--------+
|          1033|2023-08-01|       517|     2001|     157.5|     USD|
|          1017|2023-07-28|       501|     2002|    116.25|     USD|
|          1026|2023-07-30|       511|     2002|     150.0|     USD|
|          1004|2023-07-25|       503|     2002|     157.5|     USD|
|          1030|2023-07-31|       515|     2001|     127.5|     USD|
|          1034|2023-08-01|       518|     2002|     180.0|     USD|
|          1015|2023-07-27|       503|     2001|     157.5|     USD|
|          1011|2023-07-27|       501|     2002|    123.75|     USD|
|          1032|2023-07-31|       508|     2003|    191.25|     USD|
|          1024|2023-07-29|       510|     2002|      66.0|     USD|
|          1018|2023-07-28|       502|     2002|    228.75|     USD|
|          1040|2023-08-03|       

## Carregando os dados limpos e transformados em um banco de dados relacional.

In [45]:
sales_pd = sales_converted.toPandas()
website_pd = website_clean.toPandas()

In [47]:
sales_pd.columns = sales_pd.columns.str.strip()
website_pd.columns = website_pd.columns.str.strip()

#### Criando conexão ao database SQLite

In [148]:
conexao = sqlite3.connect('itau.db')

In [143]:
sales_pd.to_sql('tb_sales_data', conexao, if_exists='replace')
website_pd.to_sql('tb_website_data', conexao, if_exists='replace')

184

#### Realizando uma query no banco de dados criado

In [174]:
rows = conexao.execute('select * from tb_sales_data')
columns = [col[0] for col in rows.description]

In [175]:
sales_query = spark.createDataFrame(rows.fetchall(), schema=columns)

In [177]:
sales_query.show()

+-----+--------------+----------+----------+---------+----------+--------+
|index|transaction_id|      date|product_id|seller_id|sale_value|currency|
+-----+--------------+----------+----------+---------+----------+--------+
|    0|          1033|2023-08-01|       517|     2001|     157.5|     USD|
|    1|          1017|2023-07-28|       501|     2002|    116.25|     USD|
|    2|          1026|2023-07-30|       511|     2002|     150.0|     USD|
|    3|          1004|2023-07-25|       503|     2002|     157.5|     USD|
|    4|          1030|2023-07-31|       515|     2001|     127.5|     USD|
|    5|          1034|2023-08-01|       518|     2002|     180.0|     USD|
|    6|          1015|2023-07-27|       503|     2001|     157.5|     USD|
|    7|          1011|2023-07-27|       501|     2002|    123.75|     USD|
|    8|          1032|2023-07-31|       508|     2003|    191.25|     USD|
|    9|          1024|2023-07-29|       510|     2002|      66.0|     USD|
|   10|          1018|202

#### Fechando conexão

In [79]:
conexao.close()

## Trabalhando em uma ware-house spark

#### Salvando tabelas

In [178]:
sales_converted.write.format('orc').mode('overwrite').saveAsTable('default.sales_converted')
website_clean.write.format('orc').mode('overwrite').saveAsTable('default.website_logs')

#### Recarregando tabelas

In [179]:
sales_converted = spark.table('default.sales_converted')
website_clean = spark.table('default.website_logs')

# ANÁLISE SPARK DOS DADOS

## Identificando as páginas mais visitadas

In [184]:
website_grouped = website_clean.groupBy('page_url').count().orderBy('count', ascending=False)

In [186]:
website_grouped.show()

+-----------------+-----+
|         page_url|count|
+-----------------+-----+
|    homepage.html|   51|
|product_page.html|   46|
|    checkout.html|   35|
|     contact.html|   19|
|        blog.html|   18|
|         faq.html|   14|
|    about_us.html|    1|
+-----------------+-----+



In [205]:
website_percents = website_grouped \
    .select('*', round(((col('count')/website_grouped.select(sum('count')).collect()[0][0])*100), 2).alias('percent'))

In [207]:
website_percents.show()

+-----------------+-----+-------+
|         page_url|count|percent|
+-----------------+-----+-------+
|    homepage.html|   51|  27.72|
|product_page.html|   46|   25.0|
|    checkout.html|   35|  19.02|
|     contact.html|   19|  10.33|
|        blog.html|   18|   9.78|
|         faq.html|   14|   7.61|
|    about_us.html|    1|   0.54|
+-----------------+-----+-------+



In [222]:
# Salvando informações no ware house

website_percents.write.format('orc').mode('overwrite').saveAsTable('default.website_percents')

### As 3 páginas mais visitadas acumulam um percentual de visitas de mais de 70% de todas as visitações
#### -> homepage.html 
#### -> product_page.html 
#### -> checkout.html

## Calculando a média de duração das sessões dos usuários.

In [218]:
website_session_duration_mean = website_clean \
    .groupBy('user_id') \
    .agg(mean('session_duration')) \
    .select('user_id', round('avg(session_duration)', 2).alias('media_session_duration'))

In [220]:
website_session_duration_mean.orderBy('media_session_duration', ascending=False).show(100)

+-------+----------------------+
|user_id|media_session_duration|
+-------+----------------------+
|  10015|                 161.0|
|  10007|                152.33|
|  10002|                124.33|
|  10022|                118.75|
|  10013|                114.67|
|  10017|                104.33|
|  10006|                 99.33|
|  10027|                  96.5|
|  10009|                 87.67|
|  10026|                 82.25|
|  10033|                 81.88|
|  10035|                  76.0|
|  10029|                  70.0|
|  10037|                 69.38|
|  10039|                 67.88|
|  10014|                 62.67|
|  10028|                  58.0|
|  10034|                 57.88|
|  10018|                 57.67|
|  10010|                 54.67|
|  10031|                 52.63|
|  10032|                  52.0|
|  10003|                 48.67|
|  10024|                  48.0|
|  10036|                 46.88|
|  10021|                 45.75|
|  10038|                  44.0|
|  10040| 

In [221]:
# Quantidade de usuários distintos

website_clean.select('user_id').distinct().count()

40

In [223]:
# Salvando informações

website_session_duration_mean.write.format('orc').mode('overwrite').saveAsTable('default.website_session_duration_mean')

### São 40 usuários ao todo, a média de duração foi obtida para cada um deles e salva na tabela 'website_session_duration_mean'

## Determinando quantos usuários retornam ao site mais de uma vez por semana.

In [232]:
# Visualizando datas distintas

website_clean.select('date').distinct().orderBy('date').show(100)

+----------+
|      date|
+----------+
|2023-07-25|
|2023-07-26|
|2023-07-27|
|2023-07-28|
|2023-07-29|
|2023-07-30|
|2023-07-31|
|2023-08-01|
|2023-08-02|
|2023-08-03|
|2023-08-04|
|2023-08-05|
|2023-08-06|
|2023-08-07|
|2023-08-08|
|2023-08-09|
|2023-08-10|
|2023-08-11|
|2023-08-12|
|2023-08-13|
|2023-08-14|
|2023-08-15|
|2023-08-16|
|2023-08-17|
|2023-08-18|
|2023-08-19|
|2023-08-20|
|2023-08-21|
|2023-08-22|
|2023-08-23|
|2023-08-24|
|2023-08-25|
|2023-08-26|
|2023-08-27|
|2023-08-28|
|2023-08-29|
|2023-08-30|
|2023-08-31|
+----------+



In [269]:
# Determinando dias e semanas

tam = len(website_clean.select('date').distinct().orderBy('date').collect())

days = [website_clean.select('date').distinct().orderBy('date').collect()[i][0] for i in range(0,tam)]
weeks = [days[i] for i in range(0, tam, 7)]

In [310]:
website_weeks = website_clean \
    .withColumn('num_semana', when(col('date') <= weeks[1], lit(1)) \
               .when((col('date') > weeks[1]) & (col('date') <= weeks[2]), lit(2)) \
               .when((col('date') > weeks[2]) & (col('date') <= weeks[3]), lit(3)) \
               .when((col('date') > weeks[3]) & (col('date') <= weeks[4]), lit(4)) \
               .when((col('date') > weeks[4]) & (col('date') <= weeks[5]), lit(5)) \
               .otherwise(lit(6)))

In [311]:
website_weeks.orderBy('num_semana').show()

+-------+-----------------+----------------+----------+----------+
|user_id|         page_url|session_duration|      date|num_semana|
+-------+-----------------+----------------+----------+----------+
|  10028|        blog.html|             145|2023-07-31|         1|
|  10012|    homepage.html|              12|2023-07-27|         1|
|  10008|    homepage.html|              25|2023-07-26|         1|
|  10030|     contact.html|              28|2023-08-01|         1|
|  10022|product_page.html|              85|2023-07-29|         1|
|  10002|product_page.html|             120|2023-07-25|         1|
|  10021|product_page.html|              95|2023-07-29|         1|
|  10005|    homepage.html|              15|2023-07-31|         1|
|  10026|    about_us.html|              34|2023-07-31|         1|
|  10002|product_page.html|             130|2023-07-30|         1|
|  10001|    homepage.html|              15|2023-07-25|         1|
|  10027|     contact.html|              22|2023-07-31|       

### Juntando contagens de user_ids por semanas e fazendo a média

In [340]:
dfs = [website_weeks.where(col('num_semana') == i+1).groupBy('user_id') \
       .count().select('user_id', col('count').alias('count_semana_' + str(i+1))) 
       for i in range(0, len(weeks))]

website_freq_semanas = reduce(lambda x, y: x.join(y, 'user_id', 'full'), dfs).fillna(0)

### Contagem de visitas em cada semana para cada user_id

In [341]:
website_freq_semanas.show(40)

+-------+--------------+--------------+--------------+--------------+--------------+--------------+
|user_id|count_semana_1|count_semana_2|count_semana_3|count_semana_4|count_semana_5|count_semana_6|
+-------+--------------+--------------+--------------+--------------+--------------+--------------+
|  10001|             2|             1|             0|             0|             0|             0|
|  10002|             2|             1|             0|             0|             0|             0|
|  10003|             2|             0|             1|             0|             0|             0|
|  10004|             2|             0|             1|             0|             0|             0|
|  10005|             2|             0|             1|             0|             0|             0|
|  10006|             2|             0|             1|             0|             0|             0|
|  10007|             2|             0|             1|             0|             0|             0|


### Quantidade de user_ids que visitaram mais de 1 vez o site em ao menos 1 semana dentro do período estudado

In [373]:
website_freq_semanas.where((col('count_semana_1') > 1) |
                          (col('count_semana_2') > 1) |
                          (col('count_semana_3') > 1) |
                          (col('count_semana_4') > 1) |
                          (col('count_semana_5') > 1) |
                          (col('count_semana_6') > 1)).count()

28

### Quantidade de user_ids que visitaram mais de 1 vez, NA MÉDIA, de todas as semanas estudadas no período

In [368]:
website_freq_todas_semanas = website_freq_semanas \
    .select('user_id', ((col('count_semana_1') + col('count_semana_2') + col('count_semana_3')
                        + col('count_semana_4') + col('count_semana_5') 
                         + col('count_semana_6'))/len(weeks)).alias('media_semanas'))

In [369]:
website_freq_todas_semanas.orderBy('media_semanas', ascending=False).show()

+-------+------------------+
|user_id|     media_semanas|
+-------+------------------+
|  10030|1.3333333333333333|
|  10031|1.3333333333333333|
|  10040|1.3333333333333333|
|  10035|1.3333333333333333|
|  10032|1.3333333333333333|
|  10036|1.3333333333333333|
|  10037|1.3333333333333333|
|  10033|1.3333333333333333|
|  10038|1.3333333333333333|
|  10039|1.3333333333333333|
|  10034|1.3333333333333333|
|  10021|0.6666666666666666|
|  10022|0.6666666666666666|
|  10026|0.6666666666666666|
|  10025|0.6666666666666666|
|  10023|0.6666666666666666|
|  10027|0.6666666666666666|
|  10029|0.6666666666666666|
|  10024|0.6666666666666666|
|  10028|0.6666666666666666|
+-------+------------------+
only showing top 20 rows



In [372]:
website_freq_todas_semanas.where(col('media_semanas') > 1).count()

11

### Apenas 11 user_ids possuem uma frequência maior que 1 visita por semana, durante todo o período estudado

In [374]:
# Salvando informações

website_freq_semanas.write.format('orc').mode('overwrite').saveAsTable('default.website_freq_semanas')
website_freq_todas_semanas.write.format('orc').mode('overwrite').saveAsTable('default.website_freq_todas_semanas')

# Conclusões

### Informações sobre frequências de acesso ao site e duração de sessões foram obtidas
### Contagens realizadas
### Database criado com as informações dos CSVs, tabelas salvas.